In [ ]:
import os

In [ ]:
os.listdir()

['.config', 'drive', 'sample_data']

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, BatchNormalization

import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.models import model_from_json
from tensorflow.keras import Model as model

In [ ]:
!unzip '/content/drive/My Drive/Fruits_Dataset/fruits-360-original-size.zip' -d '/content/drive/My Drive/Fruits_Dataset/'

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/My Drive/Fruits_Dataset/fruits-360-original-size/Training/apple_red_1/54.jpg  
  inflating: /content/drive/My Drive/Fruits_Dataset/fruits-360-original-size/Training/apple_red_1/56.jpg  
  inflating: /content/drive/My Drive/Fruits_Dataset/fruits-360-original-size/Training/apple_red_1/58.jpg  
  inflating: /content/drive/My Drive/Fruits_Dataset/fruits-360-original-size/Training/apple_red_1/6.jpg  
  inflating: /content/drive/My Drive/Fruits_Dataset/fruits-360-original-size/Training/apple_red_1/60.jpg  
  inflating: /content/drive/My Drive/Fruits_Dataset/fruits-360-original-size/Training/apple_red_1/62.jpg  
  inflating: /content/drive/My Drive/Fruits_Dataset/fruits-360-original-size/Training/apple_red_1/64.jpg  
  inflating: /content/drive/My Drive/Fruits_Dataset/fruits-360-original-size/Training/apple_red_1/66.jpg  
  inflating: /content/drive/My Drive/Fruits_Dataset/fruits-360-original-size/Training/apple_re

In [ ]:
labelingjob=ImageDataGenerator(rescale=1./255)

In [ ]:
train_label=labelingjob.flow_from_directory('/content/drive/My Drive/Fruits_Dataset/Training', 
                                            target_size=(260,260), batch_size=32)  
#generate labels, makes target size of each image same

Found 5605 images belonging to 21 classes.


In [ ]:
test_label=labelingjob.flow_from_directory('/content/drive/My Drive/Fruits_Dataset/Test', 
                                            target_size=(260,260), batch_size=32)  

Found 2799 images belonging to 21 classes.


In [ ]:
type(train_label)

keras.preprocessing.image.DirectoryIterator

In [ ]:
train_label.class_indices

{'Apple rotten': 0,
 'apple6': 1,
 'apple_braeburn': 2,
 'apple_crimson_snow': 3,
 'apple_golden_1': 4,
 'apple_golden_2': 5,
 'apple_golden_3': 6,
 'apple_granny_smith': 7,
 'apple_hit': 8,
 'apple_pink_lady': 9,
 'apple_red_1': 10,
 'apple_red_2': 11,
 'apple_red_3': 12,
 'apple_red_delicios': 13,
 'apple_red_yellow': 14,
 'cabbage_white': 15,
 'carrot': 16,
 'cucumber': 17,
 'eggplant_violet': 18,
 'zucchini': 19,
 'zucchini_dark': 20}

In [ ]:
test_label.class_indices

{'Apple rotten': 0,
 'apple6': 1,
 'apple_braeburn': 2,
 'apple_crimson_snow': 3,
 'apple_golden_1': 4,
 'apple_golden_2': 5,
 'apple_golden_3': 6,
 'apple_granny_smith': 7,
 'apple_hit': 8,
 'apple_pink_lady': 9,
 'apple_red_1': 10,
 'apple_red_2': 11,
 'apple_red_3': 12,
 'apple_red_delicios': 13,
 'apple_red_yellow': 14,
 'cabbage_white': 15,
 'carrot': 16,
 'cucumber': 17,
 'eggplant_violet': 18,
 'zucchini': 19,
 'zucchini_dark': 20}

In [ ]:
valid_label=labelingjob.flow_from_directory('/content/drive/My Drive/Fruits_Dataset/Validation', 
                                            target_size=(260,260), batch_size=32)  

Found 2804 images belonging to 21 classes.


In [ ]:
Model=Sequential()

Model.add(Conv2D(32,(3,3),input_shape=(260,260,3)))   #input shape is only required at first layer
Model.add(MaxPool2D((3,3)))

for i in range(6,9):
  Model.add(Conv2D(2**i,(3,3)))     #increase filters at each iteration
  Model.add(MaxPool2D((3,3)))

Model.add(MaxPool2D(2,2))
Model.add(Flatten())

Model.add(Dense(train_label.num_classes, activation='softmax'))

In [ ]:
Model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
Model.fit(train_label, epochs=2,validation_data=valid_label)

Epoch 1/2
176/176 [==============================] - 3027s 17s/step - loss: 0.6930 - accuracy: 0.7823 - val_loss: 0.0327 - val_accuracy: 0.9957
Epoch 2/2
176/176 [==============================] - 395s 2s/step - loss: 0.1285 - accuracy: 0.9600 - val_loss: 0.0473 - val_accuracy: 0.9818


In [ ]:
Model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 258, 258, 32)      896       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 86, 86, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 84, 84, 64)        18496     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 26, 26, 128)       73856     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 6, 6, 256)        

In [ ]:
testing_path='/content/drive/My Drive/Fruits_Dataset/Test'

for i in os.listdir(testing_path):
  for j in os.listdir(os.path.join(testing_path,i))[:4]:
    my_img=load_img(os.path.join(testing_path,i,j))
    x=np.expand_dims(my_img.resize((260,260)),axis=0)
    y=Model.predict(x)
    y_class=y.argmax()
    plt.imshow(my_img)
    print('correct class for image {}'.format(i))
    print('predicted class is {}'.format(list(train_label.class_indices.keys())[y_class]))
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
Y_pred=[]
Y_act=[]

test_label.reset()
for i in range(len(test_label.filenames)):
  X,Y=test_label.next()
  Y_pred.append(Model.predict(X).argmax())
  Y_act.append(Y.argmax())

In [ ]:
test_label.class_indices.keys()

dict_keys(['Apple rotten', 'apple6', 'apple_braeburn', 'apple_crimson_snow', 'apple_golden_1', 'apple_golden_2', 'apple_golden_3', 'apple_granny_smith', 'apple_hit', 'apple_pink_lady', 'apple_red_1', 'apple_red_2', 'apple_red_3', 'apple_red_delicios', 'apple_red_yellow', 'cabbage_white', 'carrot', 'cucumber', 'eggplant_violet', 'zucchini', 'zucchini_dark'])

In [ ]:
list(test_label.class_indices.keys())

['Apple rotten',
 'apple6',
 'apple_braeburn',
 'apple_crimson_snow',
 'apple_golden_1',
 'apple_golden_2',
 'apple_golden_3',
 'apple_granny_smith',
 'apple_hit',
 'apple_pink_lady',
 'apple_red_1',
 'apple_red_2',
 'apple_red_3',
 'apple_red_delicios',
 'apple_red_yellow',
 'cabbage_white',
 'carrot',
 'cucumber',
 'eggplant_violet',
 'zucchini',
 'zucchini_dark']

In [ ]:
Y_pred

[397,
 9,
 630,
 366,
 635,
 399,
 638,
 439,
 203,
 218,
 56,
 245,
 597,
 439,
 357,
 100,
 77,
 287,
 265,
 287,
 420,
 140,
 271,
 14,
 178,
 14,
 518,
 567,
 161,
 31,
 476,
 457,
 93,
 224,
 160,
 51,
 492,
 97,
 575,
 502,
 252,
 376,
 544,
 224,
 331,
 413,
 476,
 182,
 35,
 386,
 623,
 247,
 502,
 391,
 539,
 105,
 428,
 371,
 9,
 659,
 287,
 476,
 103,
 252,
 418,
 644,
 644,
 439,
 602,
 140,
 249,
 371,
 539,
 52,
 434,
 333,
 224,
 161,
 21,
 97,
 203,
 266,
 483,
 329,
 518,
 394,
 223,
 239,
 630,
 665,
 441,
 72,
 451,
 457,
 534,
 8,
 644,
 203,
 630,
 140,
 660,
 325,
 287,
 350,
 518,
 336,
 581,
 56,
 618,
 61,
 40,
 607,
 51,
 622,
 13,
 304,
 392,
 93,
 371,
 504,
 392,
 182,
 460,
 14,
 56,
 161,
 76,
 52,
 373,
 56,
 331,
 581,
 273,
 118,
 266,
 460,
 273,
 182,
 160,
 287,
 136,
 420,
 186,
 362,
 639,
 336,
 182,
 560,
 219,
 271,
 187,
 77,
 556,
 581,
 292,
 413,
 77,
 413,
 470,
 203,
 644,
 50,
 331,
 100,
 168,
 665,
 439,
 471,
 499,
 294,
 245,
 646,
 

In [ ]:
len(Y_act)

2799

In [1]:
pred_list=[list(train_label.class_indices.keys())[i] for i in Y_pred]
act_list=[list(train_label.class_indices.keys())[i] for i in Y_act]

In [ ]:
pred_list

[['Apple rotten',
  'apple6',
  'apple_braeburn',
  'apple_crimson_snow',
  'apple_golden_1',
  'apple_golden_2',
  'apple_golden_3',
  'apple_granny_smith',
  'apple_hit',
  'apple_pink_lady',
  'apple_red_1',
  'apple_red_2',
  'apple_red_3',
  'apple_red_delicios',
  'apple_red_yellow',
  'cabbage_white',
  'carrot',
  'cucumber',
  'eggplant_violet',
  'zucchini',
  'zucchini_dark'],
 ['Apple rotten',
  'apple6',
  'apple_braeburn',
  'apple_crimson_snow',
  'apple_golden_1',
  'apple_golden_2',
  'apple_golden_3',
  'apple_granny_smith',
  'apple_hit',
  'apple_pink_lady',
  'apple_red_1',
  'apple_red_2',
  'apple_red_3',
  'apple_red_delicios',
  'apple_red_yellow',
  'cabbage_white',
  'carrot',
  'cucumber',
  'eggplant_violet',
  'zucchini',
  'zucchini_dark'],
 ['Apple rotten',
  'apple6',
  'apple_braeburn',
  'apple_crimson_snow',
  'apple_golden_1',
  'apple_golden_2',
  'apple_golden_3',
  'apple_granny_smith',
  'apple_hit',
  'apple_pink_lady',
  'apple_red_1',
  'appl

In [ ]:
len(pred_list)

2799

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(np.vstack([pred_list,act_list])).transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,5558,5559,5560,5561,5562,5563,5564,5565,5566,5567,5568,5569,5570,5571,5572,5573,5574,5575,5576,5577,5578,5579,5580,5581,5582,5583,5584,5585,5586,5587,5588,5589,5590,5591,5592,5593,5594,5595,5596,5597
0,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,...,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten,Apple rotten
1,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,...,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6,apple6
2,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,...,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn,apple_braeburn
3,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,...,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_crimson_snow,apple_

In [ ]:
#saving the model
model_json=Model.to_json()

with open("/content/drive/My Drive/Fruits_Dataset/saved_model.json", "w") as json_file:
  json_file.write(model_json)

#serialize weights to HDF5
Model.save_weights("/content/drive/My Drive/Fruits_Dataset/saved_model.h5")
print('Model saved to drive')

Model saved to drive
